# FlowsheetTools.jl Demonstration
FlowsheetTools.jl is a library for dealing with flowsheets (components, streams, unit operations, mass balance boundaries, and flowsheets).
It can be used as a platform for running custom models, for example when fitting kinetic parameters to pilot plant data, where the operating unit is more complicated than a single reactor. The primary intended purpose however, was for process analytics - generating KPIs on a flowsheet and reconciling mass balances for generic flowsheets.

For more convenient analysis of flowsheets with missing measurements, a few utility unit operations are provided: a mixer, a flow splitter, a component splitter (to emulate a separation process with split factors) and a stoichiometric reactor block with specified conversions. Custom unit operations can also be defined, by simply providing a function that calculates the outlet streams from the inlets and an optional list of parameters.

The intention was not to build a full-on process simulator, but the custom reactor blocks etc can be easily added, when needed.

Let's have a look at how to use the library.

In [1]:
using FlowsheetTools, Statistics

## Components
The most basic building block we need is a set of components. A component in FlowsheetTools.jl is a fairly simple object. It has a name, so we can refer to it, and contains the list of atoms and the number of each that make up the component. The molar mass of the component is automatically calculated and also stored.

We store all the components in a ComponentList, so we have a container to find them in, when needed. This makes things a lot easier than trying to keep track of a host of "loose" component variables. While nothing stops you from defining multiple ComponentLists, having more than one is not best practise.
In fact, to keep things consistent, the containers for streams (a `StreamList`) will insist that all streams contained in it refer to a single `ComponentList`, so it is much better to stick to one `ComponentList` at a time.

Let's start by creating our `ComponentList` to hold all of the components we have present in our flowsheet.

In [2]:
syscomps = ComponentList()

Component list:


`ComponentList` is a wrapper around a Dict{String, Component} and can be in the same way as such a `Dict`, using the component names to index. Now that we have a container to put them into, we can add some components.

Since it is likely that we shall re-use components, they can be stored in files, so we don't need to define them every time. Let's read in some components created earlier and stored in the sub-folder `components` under the active folder:

In [3]:
count = readcomponentlist!(syscomps, "components", ["Ethylene", "Ethane", "Hydrogen"])

3

The function `readcomponents` returns the number of components read in - 3 in this case. We specified the names of the components to read in from the folder. There can be any number of files stored there. We also supplied the empty component list (`syscomps`) as the first argument. This container is modified in-place.

In [4]:
syscomps

Component list:
  Ethylene
  Ethane
  Hydrogen


To access a component in the component list, we index using the name of the component.

In [5]:
syscomps["Ethylene"]

Component: Ethylene

  Atom		Count
---------------------
   C 		   2
   H 		   4


If we need to define new components, ther are two ways to go about this.

The first is by using the @comp macro. It takes a list of atoms and a number of each, separated by a -->. We also suply a name for the component, which is stored in the `ComponentList`, and the `ComponentList` to which to add it.

In [6]:
@comp begin
    N --> 2
end "Nitrogen" syscomps

Component: Nitrogen

  Atom		Count
---------------------
   N 		   2


The second way is to create the components by calling the constructor directly. This is most useful when creating lists of components, such a homologous series. For example, we could create the n-paraffins from C1 to C10 as follows:

In [7]:
cl = ComponentList()
for n in 1:10
    if n == 1
        name = "CH4"
    else
        name = "C$(n)H$(2n+2)"
    end
    cl[name] = Component(name, ["C", "H"], [n, 2n+2])
end
cl

Component list:
  CH4
  C2H6
  C3H8
  C4H10
  C5H12
  C6H14
  C7H16
  C8H18
  C9H20
  C10H22


In [8]:
cl["C10H22"]

Component: C10H22

  Atom		Count
---------------------
   C 		  10
   H 		  22


We can save the components to file to re-use later. `writecomponents` takes the path to write to, and the specific component to write. It returns the number of bytes written.

In [9]:
writecomponent(joinpath("components/", "Nitrogen.comp"), syscomps["Nitrogen"])

22

## Streams

Now that we have components (and a handy container to store them in), we can create streams for our process. Each stream contains a list of components and their flowrates. You can specify either the mass or moalr flows when creating the stream and the other will be automatically calculated. The constructor will also calculate the flowrates for each type of atom in the stream.

There are of course two ways in which we would use streams. Either with a single, current value for the flowrate and composition, or with a set of historical values of these. The former is useful for simulations, while the latter is useful for analysis. In either case, the flows are stored in `TimeArrays` from `TimeSeries.jl`. In cases where we only have a single flowrate, this is a simply `TimeArray` of length 1, with an arbitrary (zero) timestamp asigned to the value.

As was the case with components, we need a container (a stream list) to hold the streams so we have something to iterate through later. Just like `ComponentList`, `StreamList` is a wrapper around a `Dict{String, Stream}`.

In [10]:
sysstreams = StreamList()

Stream list:

Streams:
	Empty list


We can create the streams directly with instantaneous flows. This can be in either mass or molar flows. While we could call the constructor directly, using the `@stream` macro is more convenient.

The first parameter indicates whether the flows are mass or molar flows. Similarly to what we did for components with `@comp`, we then provide a list of components and their flowrates, separated with a -->. We also supply a name for the stream, against which it is stored in the `StreamList`, the component list in which the components are defined, and lastly the `StreamList` to which to add the new stream.

The units are not specified - if you assume the mass flows are in kg/h, then the molar equivalent is kmol/hr, but this could as easily be lb/week and lbmole/week.

Here we specify two stream, of identical composition and flows, but by specifying mass flows for the first and molar flows for the second.

In [11]:
@stream mass begin
    "Ethylene" --> 2.8053
    "Ethane" --> 27.06192
    "Hydrogen" --> 2.21738
end "Test" syscomps sysstreams

@stream mole begin
    "Ethane" --> 0.9
    "Hydrogen" --> 1.1
    "Ethylene" --> 0.1
end "Product" syscomps sysstreams

Stream: Product

┌─────────────┬──────────┬─────────┬──────────┬──────────┐
│             │ Ethylene │  Ethane │ Hydrogen │ Nitrogen │
├─────────────┼──────────┼─────────┼──────────┼──────────┤
│ Mass flows  │   2.8053 │ 27.0619 │  2.21738 │      0.0 │
│ Molar flows │      0.1 │     0.9 │      1.1 │      0.0 │
└─────────────┴──────────┴─────────┴──────────┴──────────┘

Total mass flow: 32.085

┌──────┬────────────┐
│ Atom │ Molar flow │
├──────┼────────────┤
│    C │        2.0 │
│    N │        0.0 │
│    H │        8.0 │
└──────┴────────────┘


One stream here was specified as mass flows, the other as molar flows, but there streams are the same and the missing flows (mass/mole) are calculated automatically in the constructor.

We can quickly check if the molar flows are identical:

In [12]:
sysstreams["Test"].moleflows .≈ sysstreams["Product"].moleflows

1×4 TimeSeries.TimeArray{Bool, 2, Dates.DateTime, BitMatrix} 0000-01-01T00:00:00 to 0000-01-01T00:00:00
┌─────────────────────┬───────────────────┬───────────────┬─────────────────────
│                     │ Ethylene_Ethylene │ Ethane_Ethane │ Hydrogen_Hydrogen  ⋯
├─────────────────────┼───────────────────┼───────────────┼─────────────────────
│ 0000-01-01T00:00:00 │              true │          true │              true  ⋯
└─────────────────────┴───────────────────┴───────────────┴─────────────────────
                                                                1 column omitted

Or, more conveniently, directly with the `≈` or `==` operators.
Keep in mind that using `==` for floating point values is likely to give `false` when you would expect `true`, so it is recommende to rather use `≈` (`\approx<tab>`)

In [13]:
sysstreams["Test"] ≈ sysstreams["Product"]

true

In [14]:
sysstreams["Test"] == sysstreams["Product"]

false

And, for the skeptical members of the audience, we can also check the atomic flows:

In [15]:
all(getindex.(values(sysstreams["Test"].atomflows), "C") .== getindex.(values(sysstreams["Product"].atomflows), "C"))

true

In [16]:
all(getindex.(values(sysstreams["Test"].atomflows), "H") .== getindex.(values(sysstreams["Product"].atomflows), "H"))

true

These comparisons also showed that we can access the flows using the `massflows`, `molarflows` and `atomflows` properties.

In [17]:
sysstreams["Product"].massflows

1×4 TimeSeries.TimeArray{Float64, 2, Dates.DateTime, Matrix{Float64}} 0000-01-01T00:00:00 to 0000-01-01T00:00:00
┌─────────────────────┬──────────┬─────────┬──────────┬──────────┐
│                     │ Ethylene │ Ethane  │ Hydrogen │ Nitrogen │
├─────────────────────┼──────────┼─────────┼──────────┼──────────┤
│ 0000-01-01T00:00:00 │   2.8053 │ 27.0619 │  2.21738 │      0.0 │
└─────────────────────┴──────────┴─────────┴──────────┴──────────┘

In [18]:
sysstreams["Product"].moleflows

1×4 TimeSeries.TimeArray{Float64, 2, Dates.DateTime, Matrix{Float64}} 0000-01-01T00:00:00 to 0000-01-01T00:00:00
┌─────────────────────┬──────────┬────────┬──────────┬──────────┐
│                     │ Ethylene │ Ethane │ Hydrogen │ Nitrogen │
├─────────────────────┼──────────┼────────┼──────────┼──────────┤
│ 0000-01-01T00:00:00 │      0.1 │    0.9 │      1.1 │      0.0 │
└─────────────────────┴──────────┴────────┴──────────┴──────────┘

In [19]:
sysstreams["Product"].atomflows

1×1 TimeSeries.TimeArray{Dict{String, Float64}, 1, Dates.DateTime, Vector{Dict{String, Float64}}} 0000-01-01T00:00:00 to 0000-01-01T00:00:00
┌─────────────────────┬────────────────────────────────────┐
│                     │ atomflows                          │
├─────────────────────┼────────────────────────────────────┤
│ 0000-01-01T00:00:00 │ Dict("C"=>2.0, "N"=>0.0, "H"=>8.0) │
└─────────────────────┴────────────────────────────────────┘

When we want to deal with streams with multiple historic data points, to analyse plant data, we can use the `readstreamhistory` function, to read the stream from a file.

First, let's start with a new, empty stream list, to get rid of the streams we have created earlier:

In [20]:
sysstreams = StreamList()

Stream list:

Streams:
	Empty list


Then we in read the streams. We need to specify the folderpath to the CSV files, the name of the stream, the component list in which the components are defined and the stream list to which to add the new stream. We also specify whether the flows in the CSV files are mass or molar flows. `ismoleflow` has a default value of false, so need not be specified for mass flows.

In [21]:
sysstreams["Feed"] = readstreamhistory(joinpath("streamhistories", "FeedStream.csv"), "Feed", syscomps; ismoleflow=true)
sysstreams["Product"] = readstreamhistory(joinpath("streamhistories", "ProdStream.csv"), "Product", syscomps; ismoleflow=true)

Stream: Product

Mass flows:
┌─────────────────────┬──────────┬─────────┬──────────┬──────────┐
│           timestamp │ Ethylene │  Ethane │ Hydrogen │ Nitrogen │
│      Dates.DateTime │  Float64 │ Float64 │  Float64 │  Float64 │
├─────────────────────┼──────────┼─────────┼──────────┼──────────┤
│ 2020-01-01T00:00:00 │  2.66076 │ 84.9202 │ 0.215866 │      0.0 │
│ 2020-01-01T06:00:00 │  2.99313 │ 93.3901 │ 0.194023 │      0.0 │
│ 2020-01-02T00:00:00 │  2.86944 │ 88.1101 │ 0.186885 │      0.0 │
│ 2020-01-02T06:00:00 │  2.98391 │ 88.3229 │  0.20051 │      0.0 │
│ 2020-01-03T00:00:00 │  3.01277 │ 92.5947 │ 0.217602 │      0.0 │
│          ⋮          │    ⋮     │    ⋮    │    ⋮     │    ⋮     │
└─────────────────────┴──────────┴─────────┴──────────┴──────────┘
                                                   22 rows omitted

Molar flows:
┌─────────────────────┬───────────┬─────────┬───────────┬──────────┐
│           timestamp │  Ethylene │  Ethane │  Hydrogen │ Nitrogen │
│      Dates.Da

In the data files (*.csv), we had columns of data for ethylene, ethane and hydrogen, but or list of components also include nitrogen.
We automatically set zero flows for amy components not in the file, so all the streams contain all of the components (for our sanity).

In the data files (*.csv), we had columns of data for ethylene, ethane and hydrogen, but our list of components also include nitrogen. We automatically set zero flows for amy components not in the file, so all the streams contain all of the components (for our sanity).

We can still add components to the component list after the streams were created. If we do, then we should also call `refreshcomplist(streamlist)` to add zero flows for all of these new components to the existing streams in the stream list.

In [22]:
@comp begin
    Ar --> 1
end "Argon" syscomps

refreshcomplist(sysstreams)

sysstreams["Feed"]

Stream: Feed

Mass flows:
┌─────────────────────┬──────────┬─────────┬──────────┬──────────┬─────────┐
│           timestamp │ Ethylene │  Ethane │ Hydrogen │ Nitrogen │   Argon │
│      Dates.DateTime │  Float64 │ Float64 │  Float64 │  Float64 │ Float64 │
├─────────────────────┼──────────┼─────────┼──────────┼──────────┼─────────┤
│ 2020-01-01T00:00:00 │  59.1012 │ 30.1711 │  3.99575 │      0.0 │     0.0 │
│ 2020-01-01T06:00:00 │  59.7961 │ 30.4876 │  4.15398 │      0.0 │     0.0 │
│ 2020-01-02T00:00:00 │  61.2996 │ 27.0851 │  4.21259 │      0.0 │     0.0 │
│ 2020-01-02T06:00:00 │  59.5127 │ 29.9669 │  3.93049 │      0.0 │     0.0 │
│ 2020-01-03T00:00:00 │  52.4608 │ 27.3167 │  4.30034 │      0.0 │     0.0 │
│          ⋮          │    ⋮     │    ⋮    │    ⋮     │    ⋮     │    ⋮    │
└─────────────────────┴──────────┴─────────┴──────────┴──────────┴─────────┘
                                                             22 rows omitted

Molar flows:
┌─────────────────────┬──────────┬──

## What can we do with streams?

Operations defined on streams include adding streams together and multiplying a stream with a scalar value. Addition of streams is effectively a mixer unit.
Multiplication is used to allow correction factors for mass balance reconciliation.

In [23]:
sysstreams["Prod2"] = 2.0*sysstreams["Product"]

Stream: Product

Mass flows:
┌─────────────────────┬──────────┬─────────┬──────────┬──────────┬─────────┐
│           timestamp │ Ethylene │  Ethane │ Hydrogen │ Nitrogen │   Argon │
│      Dates.DateTime │  Float64 │ Float64 │  Float64 │  Float64 │ Float64 │
├─────────────────────┼──────────┼─────────┼──────────┼──────────┼─────────┤
│ 2020-01-01T00:00:00 │  5.32151 │  169.84 │ 0.431731 │      0.0 │     0.0 │
│ 2020-01-01T06:00:00 │  5.98625 │  186.78 │ 0.388047 │      0.0 │     0.0 │
│ 2020-01-02T00:00:00 │  5.73888 │  176.22 │ 0.373769 │      0.0 │     0.0 │
│ 2020-01-02T06:00:00 │  5.96781 │ 176.646 │  0.40102 │      0.0 │     0.0 │
│ 2020-01-03T00:00:00 │  6.02554 │ 185.189 │ 0.435204 │      0.0 │     0.0 │
│          ⋮          │    ⋮     │    ⋮    │    ⋮     │    ⋮     │    ⋮    │
└─────────────────────┴──────────┴─────────┴──────────┴──────────┴─────────┘
                                                             22 rows omitted

Molar flows:
┌─────────────────────┬──────────

Let's check the answer:

In [24]:
sysstreams["Prod2"] == 2.0*sysstreams["Product"]

true

Alternatively,

In [25]:
all(values(sysstreams["Prod2"].totalmassflow) .== values(2.0 .* sysstreams["Product"].totalmassflow))

true

Note the use of `.==` and `.*` above. Internally the data are stored in `TimeArrays` from `TimeSeries.jl` and only the broadcasted operators are used on `TimeArray`s.

Comparison between `TimeArrays` returns a `TimeArray` with the comparison for each timestamp and we extract the results as an aray using the `values()` function to get a `BitVector`.

We can also copy streams and combine the two streams by copy with a scalar multiplication in a single call:

In [26]:
copystream!(sysstreams, "Product", "MyStream")
copystream!(sysstreams, "Product", "MyStream2"; factor=2.0) # double the flow!
sysstreams["MyStream2"] ≈ 2.0 * sysstreams["MyStream"]

true

The streams have different names, but we overload `==` to only check the molar flows of each component, so we get the expected answer.

Since the atomic flows are automatically calculated, they will also match

In [27]:
all(getindex.(values(sysstreams["Product"].atomflows), "C") .== getindex.(values(sysstreams["MyStream"].atomflows), "C"))

true

In [28]:
all(getindex.(values(sysstreams["Product"].atomflows), "H") .== getindex.(values(sysstreams["MyStream"].atomflows), "H"))

true

In [29]:
all(getindex.(values(sysstreams["Product"].atomflows), "N") .== getindex.(values(sysstreams["MyStream"].atomflows), "N"))

true

We can also rename or delete streams from the stream list:

In [30]:
renamestream!(sysstreams, "MyStream", "Dummy")
sysstreams

Stream list:

Streams:
  Feed
  Product
  Prod2
  MyStream2
  Dummy

Components:
  Ethylene
  Ethane
  Hydrogen
  Nitrogen
  Argon

Data length:	27
Data starts:	2020-01-01T00:00:00
Data ends:	2020-01-14T00:00:00


In [31]:
deletestream!(sysstreams, "Dummy")
sysstreams

Stream list:

Streams:
  Feed
  Product
  Prod2
  MyStream2

Components:
  Ethylene
  Ethane
  Hydrogen
  Nitrogen
  Argon

Data length:	27
Data starts:	2020-01-01T00:00:00
Data ends:	2020-01-14T00:00:00


# UnitOps, Boundaries and KPIs

Let's start with an empty stream list again

In [32]:
sysstreams = StreamList()

Stream list:

Streams:
	Empty list


We'll add some instantaneous flow streams.

In [33]:
@stream mole begin
    "Hydrogen" --> 1.1
end "H2" syscomps sysstreams

@stream mole begin
    "Ethylene" --> 0.1
    "Ethane" --> 0.9
end "C2" syscomps sysstreams

@stream mole begin
    "Ethylene" --> 0.0
    "Ethane" --> 1.0
    "Hydrogen" --> 1.0
end "Product" syscomps sysstreams

Stream: Product

┌─────────────┬──────────┬─────────┬──────────┬──────────┬───────┐
│             │ Ethylene │  Ethane │ Hydrogen │ Nitrogen │ Argon │
├─────────────┼──────────┼─────────┼──────────┼──────────┼───────┤
│ Mass flows  │      0.0 │ 30.0688 │   2.0158 │      0.0 │   0.0 │
│ Molar flows │      0.0 │     1.0 │      1.0 │      0.0 │   0.0 │
└─────────────┴──────────┴─────────┴──────────┴──────────┴───────┘

Total mass flow: 32.085

┌──────┬────────────┐
│ Atom │ Molar flow │
├──────┼────────────┤
│    C │        2.0 │
│    N │        0.0 │
│   Ar │        0.0 │
│    H │        8.0 │
└──────┴────────────┘


We can also add an empty stream as a placeholder. We'll calculate it with a mixer model later.

In [34]:
sysstreams["Mixed"] = emptystream(sysstreams, "Mixed")

Stream: Mixed

┌─────────────┬──────────┬────────┬──────────┬──────────┬───────┐
│             │ Ethylene │ Ethane │ Hydrogen │ Nitrogen │ Argon │
├─────────────┼──────────┼────────┼──────────┼──────────┼───────┤
│ Mass flows  │      0.0 │    0.0 │      0.0 │      0.0 │   0.0 │
│ Molar flows │      0.0 │    0.0 │      0.0 │      0.0 │   0.0 │
└─────────────┴──────────┴────────┴──────────┴──────────┴───────┘

Total mass flow: 0.0

┌──────┬────────────┐
│ Atom │ Molar flow │
├──────┼────────────┤
│    C │        0.0 │
│    N │        0.0 │
│   Ar │        0.0 │
│    H │        0.0 │
└──────┴────────────┘


Now we define some unit operations. As with components and streams we create a container to be able to conveniently access them again later. A `UnitOpList` works the same way as a `ComponentList` or `StreamList` - it is a wrapper around a `Dict{String, UnitOp}`.

In [35]:
sysunitops = UnitOpList()

@unitop begin
    inlets --> ["H2", "C2"]
    outlets --> ["Mixed"]
    calc --> mixer!
end "Mixer" sysstreams sysunitops

Unit Operation:  Mixer
Feed streams:    ["H2", "C2"]

Product streams: ["Mixed"]


The `@unitop` macro creates a `UnitOp` object and adds it to the `UnitOpList`. We can then refer to it by its name, like we do with `Component` and `Stream` objects.
The macro takes an array of `Stream` names for inlets, and another for outlets. The `calc` field is optional. If we are only calculating KPIs for our process or reconciling mass balances, unit operations do not need to do calculations. They only serve as nodes where streams are connected.
If we want to do calculations, like for mixers and splitters, we need to specifiy the name of the function to call in the `calc` field.
The function takes the form:
    function functionname!(streamlist::StreamList, outlets::Vector{String}, inlets::Vector{String}, params)
As per Julia convention, we add a `!` at the end of the function name, which means the function modifies some of the variables passed - the outlet(s).
There can be multiple inlets and outlets, but even single inlets and outlets must be passed in an array.
All streams passed to a unit operation must be in the stream list. And all streams in a stream list must use the same component list. This keeps things consistent.
The `params` field does not have a defined type and can be anything needed for the calculation. We'll look at this in more detail later.

To execute the unit operation, we simply call it.

In [36]:
sysunitops["Mixer"]()

This `UnitOp` takes the required inlet and outlet streams, but is also assigned a calculation.
In this case, it is the predefined `mixer!` function, which is a simple stream mixer.
This can however be any user-defined function, with the correct form.
These calculations will supply the contents of the outlet streams based on the inlets streams and supplied model parameters.
They are only needed if there is no information on the outlet streams.

In [37]:
@unitop begin
    inlets --> ["Mixed"]
    outlets --> ["Product"]
end "Reactor" sysstreams sysunitops

Unit Operation:  Reactor
Feed streams:    ["Mixed"]

Product streams: ["Product"]


Our `Reactor` does not have an associated calculation. It is just a node in the flowsheet graph, so we shall need information for all of the inlets and outlets.

Let's split the product a little. We'll need some empty streams.

In [38]:
sysstreams["Product1"] = emptystream(sysstreams, "Product1");
sysstreams["Product1a"] = emptystream(sysstreams, "Product1a");
sysstreams["Product1b"] = emptystream(sysstreams, "Product1b");
sysstreams["Product2"] = emptystream(sysstreams, "Product2");
sysstreams["Product3"] = emptystream(sysstreams, "Product3");

Let's also add a dummy stream with a fixed composition. We don't need it here, but it is easy to do

In [39]:
sysstreams["Dummy"] = fixedstream(sysstreams, "Dummy", [10.0, 0.0, 0.0, 0.0, 0.1])

Stream: Dummy

┌─────────────┬──────────┬────────┬──────────┬──────────┬────────────┐
│             │ Ethylene │ Ethane │ Hydrogen │ Nitrogen │      Argon │
├─────────────┼──────────┼────────┼──────────┼──────────┼────────────┤
│ Mass flows  │     10.0 │    0.0 │      0.0 │      0.0 │        0.1 │
│ Molar flows │ 0.356468 │    0.0 │      0.0 │      0.0 │ 0.00250325 │
└─────────────┴──────────┴────────┴──────────┴──────────┴────────────┘

Total mass flow: 10.1

┌──────┬────────────┐
│ Atom │ Molar flow │
├──────┼────────────┤
│    C │   0.712936 │
│    N │        0.0 │
│   Ar │ 0.00250325 │
│    H │    1.42587 │
└──────┴────────────┘


A flow splitter that splits 50% of the product to each of Product1 and Product2. These streams will have identcal compositions

In [40]:
@unitop begin
    inlets --> ["Product"]
    outlets --> ["Product1", "Product2"]
    calc --> flowsplitter!
    params --> [0.5]
end "ProductSplitter" sysstreams sysunitops

sysunitops["ProductSplitter"]()

A component splitter that splits Product1 into Product1a and Product1b.
These streams will have different compositions, with the hydrogen split 50:50, 70% of the ethane going to Product1b and the remainder of Product1, going to Product1b (the last stream listed).

In [41]:
@unitop begin
    inlets --> ["Product1"]
    outlets --> ["Product1a", "Product1b"]
    calc --> componentplitter!
    params --> Dict([
        "Hydrogen" => Dict(["Product1a" => 0.5]),
        "Ethane" => Dict(["Product1b" => 0.3])
    ])
end "ComponentSplitter" sysstreams sysunitops

sysunitops["ComponentSplitter"]()

And then we mix it all again and check that we still have the original Product stream

In [42]:
@unitop begin
    inlets --> ["Product1a", "Product1b", "Product2"]
    outlets --> ["Product3"]
    calc --> mixer!
end "Mixer2" sysstreams sysunitops

sysunitops["Mixer2"]()

Check that the two streams have the same flows

In [43]:
sysstreams["Product"] ≈ sysstreams["Product3"]

true

Mass balances and KPIs are defined on a boundary around a number of unit operations. We therefore define a `Boundary` and list the contained `UnitOp`s

In [44]:
sysboundaries = BoundaryList()

@boundary begin
    unitops --> ["Mixer", "Reactor"]
end "B1" sysunitops sysboundaries

Balance Boundary:

Enclosed units: ["Mixer", "Reactor"]

Closure:
┌─────────────────────┬────────────────────┐
│           timestamp │ Total Mass Closure │
│      Dates.DateTime │            Float64 │
├─────────────────────┼────────────────────┤
│ 0000-01-01T00:00:00 │                1.0 │
└─────────────────────┴────────────────────┘

Combined Feed Mass Flows:
┌─────────────────────┬──────────┬─────────┬──────────┬──────────┬─────────┐
│           timestamp │ Ethylene │  Ethane │ Hydrogen │ Nitrogen │   Argon │
│      Dates.DateTime │  Float64 │ Float64 │  Float64 │  Float64 │ Float64 │
├─────────────────────┼──────────┼─────────┼──────────┼──────────┼─────────┤
│ 0000-01-01T00:00:00 │   2.8053 │ 27.0619 │  2.21738 │      0.0 │     0.0 │
└─────────────────────┴──────────┴─────────┴──────────┴──────────┴─────────┘
Combined Product Mass Flows:
┌─────────────────────┬──────────┬─────────┬──────────┬──────────┬─────────┐
│           timestamp │ Ethylene │  Ethane │ Hydrogen │ Nitrogen │   

We can look at total mass and elemental closures, as well as the combined in- and outflows.

In [45]:
sysboundaries["B1"].atomclosures

1×1 TimeSeries.TimeArray{Dict{String, Float64}, 1, Dates.DateTime, Vector{Dict{String, Float64}}} 0000-01-01T00:00:00 to 0000-01-01T00:00:00
┌─────────────────────┬───────────────────────────────────────────────┐
│                     │ Elemental Closures                            │
├─────────────────────┼───────────────────────────────────────────────┤
│ 0000-01-01T00:00:00 │ Dict("C"=>1.0, "N"=>0.0, "Ar"=>0.0, "H"=>1.0) │
└─────────────────────┴───────────────────────────────────────────────┘

In [46]:
sysboundaries["B1"].closure

1×1 TimeSeries.TimeArray{Float64, 1, Dates.DateTime, Vector{Float64}} 0000-01-01T00:00:00 to 0000-01-01T00:00:00
┌─────────────────────┬────────────────────┐
│                     │ Total Mass Closure │
├─────────────────────┼────────────────────┤
│ 0000-01-01T00:00:00 │                1.0 │
└─────────────────────┴────────────────────┘

In [47]:
sysboundaries["B1"].total_in.totalmassflow

1×1 TimeSeries.TimeArray{Float64, 1, Dates.DateTime, Vector{Float64}} 0000-01-01T00:00:00 to 0000-01-01T00:00:00
┌─────────────────────┬────────────────┐
│                     │ totalmassflows │
├─────────────────────┼────────────────┤
│ 0000-01-01T00:00:00 │        32.0846 │
└─────────────────────┴────────────────┘

In [48]:
sysboundaries["B1"].total_out.totalmassflow

1×1 TimeSeries.TimeArray{Float64, 1, Dates.DateTime, Vector{Float64}} 0000-01-01T00:00:00 to 0000-01-01T00:00:00
┌─────────────────────┬────────────────┐
│                     │ totalmassflows │
├─────────────────────┼────────────────┤
│ 0000-01-01T00:00:00 │        32.0846 │
└─────────────────────┴────────────────┘

We can also define KPIs on the boundary. Here we use the pre-defined KPIs of `conversion(boundary, component)` and `selectivity(boundary, reactant, product)`

In [49]:
conversion(sysboundaries["B1"], "Ethane")

-0.11111111111111106

Ethane was produced, not consumed, so has a negative value for conversion.

In [50]:
(conversion(sysboundaries["B1"], "Ethylene"), conversion(sysboundaries["B1"], "Hydrogen"))

(1.0, 0.09090909090909104)

We had complete conversion of ethylene and only ~9% of hydrogen, due to the large excess fed.

In [51]:
molar_selectivity(sysboundaries["B1"], "Ethylene", "Ethane")

0.9999999999999998

All of the reacted ethylene was converted to ethane.

### For streams with time series data

Now we can repeat this for streams with multiple historic data points attached:

In [52]:
sysstreams = StreamList() # Create a new container and dump the previous streams
sysstreams["C2"] = readstreamhistory(joinpath("streamhistories", "C2.csv"), "C2", syscomps; ismoleflow=true)
sysstreams["H2"] = readstreamhistory(joinpath("streamhistories", "Hydrogen.csv"), "H2", syscomps; ismoleflow=true)
sysstreams["Product"] = readstreamhistory(joinpath("streamhistories", "Product.csv"), "Product", syscomps; ismoleflow=true)
sysstreams["Mixed"] = emptystream(sysstreams, "Mixed");
sysstreams["Product1"] = emptystream(sysstreams, "Product1");
sysstreams["Product1a"] = emptystream(sysstreams, "Product1a");
sysstreams["Product1b"] = emptystream(sysstreams, "Product1b");
sysstreams["Product2"] = emptystream(sysstreams, "Product2");
sysstreams["Product3"] = emptystream(sysstreams, "Product3");

And just to show what happens when we create a fixed composition stream when the streams in the StreamList have time series data

In [53]:
sysstreams["Dummy"] = fixedstream(sysstreams, "Dummy", [10.0, 0.0, 0.0, 0.0, 0.1])

Stream: Dummy

Mass flows:
┌─────────────────────┬──────────┬─────────┬──────────┬──────────┬─────────┐
│           timestamp │ Ethylene │  Ethane │ Hydrogen │ Nitrogen │   Argon │
│      Dates.DateTime │  Float64 │ Float64 │  Float64 │  Float64 │ Float64 │
├─────────────────────┼──────────┼─────────┼──────────┼──────────┼─────────┤
│ 2020-01-01T00:00:00 │     10.0 │     0.0 │      0.0 │      0.0 │     0.1 │
│ 2020-01-01T06:00:00 │     10.0 │     0.0 │      0.0 │      0.0 │     0.1 │
│ 2020-01-02T00:00:00 │     10.0 │     0.0 │      0.0 │      0.0 │     0.1 │
│ 2020-01-02T06:00:00 │     10.0 │     0.0 │      0.0 │      0.0 │     0.1 │
│ 2020-01-03T00:00:00 │     10.0 │     0.0 │      0.0 │      0.0 │     0.1 │
│          ⋮          │    ⋮     │    ⋮    │    ⋮     │    ⋮     │    ⋮    │
└─────────────────────┴──────────┴─────────┴──────────┴──────────┴─────────┘
                                                             22 rows omitted

Molar flows:
┌─────────────────────┬──────────┬─

Empty the unit operation list as well, so we start fresh.

In [54]:
sysunitops = UnitOpList()

@unitop begin
    inlets --> ["H2", "C2"]
    outlets --> ["Mixed"]
    calc --> mixer!
end "Mixer" sysstreams sysunitops
sysunitops["Mixer"]()

@unitop begin
    inlets --> ["Mixed"]
    outlets --> ["Product"]
end "Reactor" sysstreams sysunitops

@unitop begin
    inlets --> ["Product"]
    outlets --> ["Product1", "Product2"]
    calc --> flowsplitter!
    params --> [0.5]
end "ProductSplitter" sysstreams sysunitops
sysunitops["ProductSplitter"]()

@unitop begin
    inlets --> ["Product1"]
    outlets --> ["Product1a", "Product1b"]
    calc --> componentplitter!
    params --> Dict([
        "Hydrogen" => Dict(["Product1a" => 0.5]),
        "Ethane" => Dict(["Product1b" => 0.3])
    ])
end "ComponentSplitter" sysstreams sysunitops
sysunitops["ComponentSplitter"]()

@unitop begin
    inlets --> ["Product1a", "Product1b", "Product2"]
    outlets --> ["Product3"]
    calc --> mixer!
end "Mixer2" sysstreams sysunitops
sysunitops["Mixer2"]()

Check that the two streams have the same flows.

In [55]:
sysstreams["Product"] ≈ sysstreams["Product3"]

true

Define the mass balance boundary for closures and KPIs

In [56]:
sysboundaries = BoundaryList()
@boundary begin
    unitops --> ["Mixer", "Reactor"]
end "B1" sysunitops sysboundaries

sysboundaries["B1"].atomclosures

sysboundaries["B1"].closure

sysboundaries["B1"].total_in.totalmassflow

sysboundaries["B1"].total_out.totalmassflow

c1 = conversion(sysboundaries["B1"], "Ethane")
c2 = conversion(sysboundaries["B1"], "Ethylene")

sc2 = molar_selectivity(sysboundaries["B1"], "Ethylene", "Ethane")

(mean(values(c1)), mean(values(c2)), mean(values(sc2)))

(-0.11361804202204312, 0.9988638054742306, 0.9999999999999998)

So, we have average conversions of ethane (-11%, meaning it was produced, not consumed), ethylene (99.9%) and selectivity of ethylene conversion to ethane (~100%) similar to the single data point above.

## Mass balance reconciliation

The mass balance reconciliation algorithm is currently *VERY BASIC*! This will be updated at the first opportunity, but will be invisible to the end-user and will not have major impacts on the user interface unless additional user input is required.

To demomstrate the use of the reconciliation tool, we repeat the flowsheet above, but introduce some (artificial) flow measurement errors.

In [57]:
copystream!(sysstreams, "C2", "eC2", factor = 1.05)
copystream!(sysstreams, "H2", "eH2", factor = 0.95)
copystream!(sysstreams, "Product", "eProduct")
sysstreams["eMixed"] = emptystream(sysstreams, "eMixed") # We'll calculate this stream with the mixer model

@unitop begin
    inlets --> ["eH2", "eC2"]
    outlets --> ["eMixed"]
    calc --> mixer!
end "eMixer" sysstreams sysunitops
sysunitops["eMixer"]()

@unitop begin
    inlets --> ["eMixed"]
    outlets --> ["eProduct"]
end "eReactor" sysstreams sysunitops

@boundary begin
    unitops --> ["eMixer", "eReactor"]
end "B2" sysunitops sysboundaries

Balance Boundary:

Enclosed units: ["eMixer", "eReactor"]

Closure:
┌─────────────────────┬────────────────────┐
│           timestamp │ Total Mass Closure │
│      Dates.DateTime │            Float64 │
├─────────────────────┼────────────────────┤
│ 2020-01-01T00:00:00 │           0.958788 │
│ 2020-01-01T06:00:00 │           0.958655 │
│ 2020-01-02T00:00:00 │           0.958268 │
│ 2020-01-02T06:00:00 │           0.958248 │
│ 2020-01-03T00:00:00 │           0.958608 │
│          ⋮          │         ⋮          │
└─────────────────────┴────────────────────┘
                             22 rows omitted

Combined Feed Mass Flows:
┌─────────────────────┬──────────┬─────────┬──────────┬──────────┬─────────┐
│           timestamp │ Ethylene │  Ethane │ Hydrogen │ Nitrogen │   Argon │
│      Dates.DateTime │  Float64 │ Float64 │  Float64 │  Float64 │ Float64 │
├─────────────────────┼──────────┼─────────┼──────────┼──────────┼─────────┤
│ 2020-01-01T00:00:00 │   3.1588 │ 28.1775 │  2.13952 │  

We can request the correction factors, without applying them.

In [58]:
corrections = calccorrections(sysboundaries)

Dict{String, Float64} with 6 entries:
  "C2"       => 1.0
  "Product"  => 1.0
  "eC2"      => 0.95095
  "eH2"      => 1.04648
  "eProduct" => 0.998
  "H2"       => 1.0

We can also pass through a custom additional error function to minimize.
We have a custom function that receives a dictionary of the correction factors and can do any arbitrary calculation.
The retured value is added to the weighted square errors of the total mass and element balances.
You can, for example, add distance from equilibrium for a reaction to the reconciliation.

In [59]:
myerr(factorDict) = 100.0 * sum(abs2, 1.0 .- values(factorDict))
corrections = calccorrections(sysboundaries, customerror=myerr)

Dict{String, Float64} with 6 entries:
  "C2"       => 1.0
  "Product"  => 1.0
  "eC2"      => 0.988278
  "eH2"      => 0.999576
  "eProduct" => 1.01187
  "H2"       => 1.0

The previous example uses a constant weight for all elements, but we can specifiy individual weights as well.

In [60]:
weights = Dict(["H" => 1.0, "C" => 1.5, "O" => 1.0, "Ar" => 0.0, "N" => 0.0])
corrections2 = calccorrections(sysboundaries, customerror=myerr, setelements=true, elementweights=weights)

Dict{String, Float64} with 6 entries:
  "C2"       => 1.0
  "Product"  => 1.0
  "eC2"      => 0.986905
  "eH2"      => 0.99976
  "eProduct" => 1.01299
  "H2"       => 1.0

`calccorrections` takes a boundary for which to calculate the correction factors, and then optional weights for the total mass balance error and the elemental errors.
These latter values default to 1.0 each. It uses [Ridge Regression](https://www.ibm.com/topics/ridge-regression?_sm_au_=iF5HM658VZnjn6Sr0GLqHKHB1jtC6) with a default λ = 0.1

  function calccorrections_anchor(boundary::BalanceBoundary, anchor::String; totalweight=1.0, elementweight = 1.0,
        setelements = false, elementweights::Dict{String, Float64} = Dict{String, Float64}())

An alternative option is to use an anchor stream, rather than Ridge Regression. The correction factor for the anchor stream will be 1.0, i.e. there is no adjustment.

corrections_a = calccorrections_anchor(sysboundaries["B2"], "eProduct")
corrections_a = calccorrections_anchor(sysboundaries["B2"], "eProduct", myerr)

Or again, with inidividual element weights

corrections_a = calccorrections_anchor(sysboundaries["B2"], "eProduct", setelements=true, elementweights=weights)

`calccorrections_anchor` takes a boundary for which to calculate the correction factors, an anchor stream, for which the correction is always 1.0 - no change, and then optional weights for the total mass balance error and the elemental errors.
These latter values default to 1.0 each.

  `function calccorrections(boundary::BalanceBoundary, anchor::String; totalweight=1.0, elementweight=1.0)`

We can apply the corrections, with `closemb()`, which will take a `Dict` of correction factors.

b2 = closemb(sysboundaries["B2"], corrections)
b3 = closemb(sysboundaries["B2"], corrections_a)

Let's compare the raw and reconciled closures:

(mean(values(b.closure)), mean(values(b2.closure)), mean(values(b3.closure)))

We can now write corrected streams back to file

writestreamhistory(sysstreams["C2"], "corrected.csv")

We can also request some information from a bounary. This is given in table form, packed into a string.

#  nb print(showdata(b2))

## Flowsheets

Lastly, for convenience, we can creat a `Flowsheet` object, which holds a number of unit operations and an execution order.
If the flowsheet is then executed, each unit operation is execute in order, as specified.
Unit operations can be added or deleted with utility functions and the execution order can be modified.

fs = Flowsheet(sysunitops, ["Reactor"], [1])
addunitop!(fs, ["Mixer", "ProductSplitter", "ComponentSplitter", "Mixer2"])

#nb fs()

Lastly, once a `Flowsheet` object is created, a block flow diagram can also be generated.

#nb generateBFD(fs, "./myflowsheet.svg")

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*